In [2]:
import pandas as pd
import os
import scipy.stats
import numpy as np
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
#from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
%matplotlib inline
import tqdm
#from sklearn.metrics import average_precision_score
#from sklearn.metrics import precision_recall_curve
#import matplotlib.pyplot as plt
#from inspect import signature
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [172]:
path="dataconv_GMQL/res/files/"

listdf_r=[]

for fname in os.listdir(path):
    if fname.endswith(".meta"): 
        pass
    elif fname.endswith(".gdm"): 
        
        tempdf=pd.read_csv(path+fname, delimiter="\t", header=None)
        
        tempdf["genes"]=tempdf[4]+"-"+tempdf[6]
        tempdf["fpkm-uq"]=tempdf[10]
        tempdf=tempdf[["genes", "fpkm-uq"]]
        tempdf["fname"]=fname
        
        listdf_r.append(tempdf)

In [173]:
path="dataconv_GMQL/sen_l/files/"

listdf_sl=[]

for fname in os.listdir(path):
    if fname.endswith(".meta"): 
        pass
    elif fname.endswith(".gdm"): 
        
        tempdf=pd.read_csv(path+fname, delimiter="\t", header=None)
        
        tempdf["genes"]=tempdf[4]+"-"+tempdf[6]
        tempdf["fpkm-uq"]=tempdf[10]
        tempdf=tempdf[["genes", "fpkm-uq"]]
        tempdf["fname"]=fname
        
        listdf_sl.append(tempdf)

In [174]:
path="dataconv_GMQL/sen_s/files/"

listdf_ss=[]

for fname in os.listdir(path):
    if fname.endswith(".meta"): 
        pass
    elif fname.endswith(".gdm"): 
        
        tempdf=pd.read_csv(path+fname, delimiter="\t", header=None)
        
        tempdf["genes"]=tempdf[4]+"-"+tempdf[6]
        tempdf["fpkm-uq"]=tempdf[10]
        tempdf=tempdf[["genes", "fpkm-uq"]]
        tempdf["fname"]=fname
        
        listdf_ss.append(tempdf)

In [175]:
totaldf_ss=pd.concat(listdf_ss)
totaldf_ss.columns=["gene", "s_value", "patient"]

In [176]:
totaldf_sl=pd.concat(listdf_sl)
totaldf_sl.columns=["gene", "s_value", "patient"]

In [177]:
totaldf_r=pd.concat(listdf_r)
totaldf_r.columns=["gene", "s_value", "patient"]

In [178]:
pivoteddf_ss=totaldf_ss.pivot(index="patient", columns="gene", values="s_value")
#pivoteddf_ss.head()

In [179]:
pivoteddf_sl=totaldf_sl.pivot(index="patient", columns="gene", values="s_value")
#pivoteddf_sl.head()

In [180]:
pivoteddf_r=totaldf_r.pivot(index="patient", columns="gene", values="s_value")
#pivoteddf_r.head()

trasposta=np.transpose(pivoteddf_sl)

genes=[]

for x in ["-".join(x.strip().split("-")[1:])  for x in (trasposta.index)]:
    genes.append(x)  

trasposta['genes']=genes
sel_col=[f for f in trasposta.columns if ("gene" in f or "genes"in f)]
trasposta=trasposta[sel_col]
#trasposta
trasposta.to_csv('genes_list_sl.csv', sep='\t')

In [181]:
list_pv=[]


for g in (pivoteddf_sl.columns): 
    c1=pivoteddf_sl[g].values
    c2=pivoteddf_ss[g].values
    m1=np.median(c1)
    m2=np.median(c2)
    
    if m1==0.0 and m2==0.0:
        pass
    else:
    
        try: 

            #pv=scipy.stats.ttest_ind(c1,c2).pvalue
            pv=scipy.stats.mannwhitneyu(c1,c2).pvalue
            if pv<0.001:
                list_pv.append((g,pv,m1,m2))
                #print(g,pv,m1,m2)

        except:
            pass

In [182]:
results=sorted(list_pv, key=lambda x:x[1])
len(results)


#for h in ["\t".join(map (str,list(x))) for x in results]:
     
dataset_result=pd.DataFrame(results)
#results

In [183]:
dataset_result["sum"]=dataset_result[2]+dataset_result[3]
new_ds=dataset_result[dataset_result["sum"]>10]
#new_ds.shape
new_ds.columns=['ensemble_name', 'p_value', 'median_r', 'median_sl', 'sum']

In [184]:
names=[]

for x in ["-".join(x.strip().split("-")[1:])  for x in (new_ds['ensemble_name'])]:
    names.append(x)    

In [185]:
new_ds['gene_symbol']=names
new_ds
#new_ds.to_csv('genes_r_vs_sl.csv', sep='\t')

/home/lalli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,ensemble_name,p_value,median_r,median_sl,sum,gene_symbol
2,ENSG00000138668.17-HNRNPD,0.000039,35.138298,25.373630,60.511928,HNRNPD
17,ENSG00000067182.6-TNFRSF1A,0.000211,34.266740,45.816780,80.083519,TNFRSF1A
34,ENSG00000145623.11-OSMR,0.000344,4.848177,7.763813,12.611990,OSMR
36,ENSG00000250742.1-RP11-834C11.4,0.000388,30.731413,13.888525,44.619938,RP11-834C11.4
37,ENSG00000164379.5-FOXQ1,0.000397,6.644775,19.918736,26.563511,FOXQ1
40,ENSG00000196937.9-FAM3C,0.000437,13.300779,17.102736,30.403516,FAM3C
54,ENSG00000115053.14-NCL,0.000620,91.403944,64.435945,155.839889,NCL
59,ENSG00000132002.6-DNAJB1,0.000680,35.312594,51.167024,86.479618,DNAJB1
69,ENSG00000156804.6-FBXO32,0.000780,3.914194,9.061693,12.975887,FBXO32
71,ENSG00000188010.11-MORN2,0.000797,18.670163,12.358586,31.028749,MORN2


In [186]:
column=(dataset_result[0])
#column=list(dataset_result[0])

In [187]:
pivoteddf_ss_selected=pivoteddf_ss[column]
pivoteddf_ss_selected["label"]=0
pivoteddf_ss_selected.shape

/home/lalli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(111, 104)

In [188]:
pivoteddf_sl_selected=pivoteddf_sl[column]
pivoteddf_sl_selected["label"]=1
pivoteddf_sl_selected.shape

/home/lalli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(19, 104)

In [189]:
concatenated=pd.concat([pivoteddf_sl_selected, pivoteddf_ss_selected])

In [190]:
X=concatenated[concatenated.columns[:-1]].values
y=concatenated[concatenated.columns[-1]].values

In [191]:
clf = RandomForestClassifier(n_estimators=10, max_depth=4, random_state=0)

In [192]:
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [193]:
clf.predict(X)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [194]:
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=20, random_state=36851234)

In [195]:
accuracy=[]
precision=[]
recall=[]

for train_index, test_index in (rskf.split(X, y)) :
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]
    clf = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0)
    #clf = svm.SVC(kernel='rbf')
    clf.fit(X_train, y_train)
    y_predict=clf.predict(X_test)
    
    accuracy.append(accuracy_score(y_test, y_predict))
    precision.append(precision_score(y_test, y_predict))
    recall.append(recall_score(y_test, y_predict))

/home/lalli/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lalli/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lalli/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lalli/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lalli/anaconda3/lib/python3.6/site-packages/sklear

/home/lalli/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lalli/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lalli/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lalli/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lalli/anaconda3/lib/python3.6/site-packages/sklear

In [196]:
np.mean(accuracy) #accuratezza
#print(accuracy_score(y_test, y_predict))

0.8733299145299145

In [197]:
np.mean(precision)

0.43

In [198]:
np.mean(recall)

0.13083333333333333

In [ ]:
#common_genes_clintab
dset1=pd.read_csv ("clintab_GMQL/genes_res_vs_sl.csv", delimiter='\t', index_col=0)
dset2=pd.read_csv ("clintab_GMQL/genes_res_vs_ss.csv", delimiter='\t', index_col=0)
dset3=pd.read_csv ("clintab_GMQL/genes_sl_vs_ss.csv", delimiter='\t', index_col=0)

lista1=set(dset1['gene_symbol'])
lista2=set(dset2['gene_symbol'])
lista3=set(dset3['gene_symbol'])

lista1.intersection(lista2)

def remove_duplicates():
    t = [lista1]
    t2 = [lista2]
    for t in t2:
        t.append(t.remove())
    return t

In [6]:
dset1=pd.read_csv("geni_Res_SenL.csv", delimiter=',')
dset2=pd.read_csv("geni_Res_SenS.csv", delimiter=',')
dset3=pd.read_csv("geni_SenL_SenS.csv", delimiter=',')

In [7]:
lista1=set(dset1['gene_symbol'])
lista2=set(dset2['gene_symbol'])
lista3=set(dset3['gene_symbol'])

In [10]:
lista2.intersection(lista3)

{'EPAS1', 'RPL36', 'RPS24'}